In [1]:
# 자동화하기
# 이어서 쓸 때 save_to_file과 append_to_file 함수에 mode의 파라미터를 'a'로 변경해보기
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
from selenium.webdriver.chrome.options import Options
import csv

In [2]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome()
# 설치 폴더에 주의합니다. 

driver.get('https://logins.daum.net/accounts/loginform.do?mobilefull=1&category=cafe&url=http%3A%2F%2Fm.cafe.daum.net%2F_myCafe%3Fnull')
# 19년 5월부터 로그인 페이지 주소가 살짝 바뀌었네요. 

time.sleep(3)


Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36


In [3]:
#예외 처리
CAFE_NAME = 'rocksoccer'
BOARD_NAME = 'ADs2'
QUERY = '쿠플'
driver.get(f'http://m.cafe.daum.net/{CAFE_NAME}/{BOARD_NAME}/search?r=https%3A%2F%2Fm.cafe.daum.net%2Frocksoccer%2FADs2&query={QUERY}')
time.sleep(3)

In [4]:
# 마지막 페이지까지 클릭
def go_to_last_page():
    wait = WebDriverWait(driver, 10)
    while True:
        try:
            # '다음 페이지' 버튼을 찾기
            next_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.btn_page.btn_next")))
            next_button_text = next_button.text.strip()
            # 버튼이 비활성화 상태인지 확인
            if next_button_text == '다음 목록이 없습니다.':
                print("Reached the last page.")
                break

            # '다음 페이지' 버튼 클릭
            next_button.click()
            # 로딩 시간을 기다리기 위해 sleep 사용
            time.sleep(5)
        except TimeoutException:
            print("Reached the last page.")
            break

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup
# 마지막 페이지 번호 알아내기
def get_last_page():
    soup = go_to_last_page()
    page_list = []
    # 모든 'num_page' 클래스를 가진 span 태그 내에서 페이지 번호 추출
    for span in soup.find_all("span", class_="num_page"):
        # 'link_page' 클래스를 가진 태그(a 태그 또는 em 태그) 찾기
        page_tag = span.find(class_="link_page")
        if page_tag:
            # 페이지 번호를 리스트에 추가
            page_list.append(page_tag.get_text(strip=True))
    max_page = page_list[-1]
    print(f"총 {max_page}개의 페이지가 있습니다")
    return max_page

# 페이지 링크 모두 가져오기
def get_boards(page_num):
    boards=[]
    for page in range(page_num):
        boards.append(f"https://m.cafe.daum.net/{CAFE_NAME}/{BOARD_NAME}/search/all?query={QUERY}&r=https%3A%2F%2Fm.cafe.daum.net%2F{CAFE_NAME}%2F{BOARD_NAME}&sort=0&head=&page={page+1}")
    return boards

# 게시글 링크 가져오기
def get_posts():
    global QUERY
    global PAGES
    board_links = get_boards(PAGES)
    posts = []
    for board_link in board_links:
        req = requests.get(board_link)
        print(req.status_code)
        soup = BeautifulSoup(req.text, 'html.parser')
        tds = soup.find_all('a', {'class':'link_cafe #search_result_list'})
        for td in tds:
            if td is not None:
               posts.append(td['href'])
    print(f"총 {len(posts)}개의 글 링크를 찾았습니다")
    
    # 게시글 링크 csv로 저장
    post_file = open(f"ROCKSOCCER_{QUERY}_{PAGES}pages_inner_links.csv", mode='w', encoding='utf-8')
    writer = csv.writer(post_file)
    for post in posts:
        writer.writerow([post])
    post_file.close()
    return posts


# 크롤링할 페이지 수 설정
PAGES = int(get_last_page())
# 게시글 정보 크롤링 및 저장
get_posts()


Reached the last page.
총 23개의 페이지가 있습니다


NameError: name 'get_contents' is not defined

In [5]:
get_posts()

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
총 458개의 글 링크를 찾았습니다


['/rocksoccer/ADs2/489458?searchView=Y',
 '/rocksoccer/ADs2/489439?searchView=Y',
 '/rocksoccer/ADs2/488900?searchView=Y',
 '/rocksoccer/ADs2/488895?searchView=Y',
 '/rocksoccer/ADs2/488622?searchView=Y',
 '/rocksoccer/ADs2/488441?searchView=Y',
 '/rocksoccer/ADs2/488419?searchView=Y',
 '/rocksoccer/ADs2/488418?searchView=Y',
 '/rocksoccer/ADs2/488416?searchView=Y',
 '/rocksoccer/ADs2/488229?searchView=Y',
 '/rocksoccer/ADs2/485529?searchView=Y',
 '/rocksoccer/ADs2/485523?searchView=Y',
 '/rocksoccer/ADs2/485095?searchView=Y',
 '/rocksoccer/ADs2/484607?searchView=Y',
 '/rocksoccer/ADs2/483610?searchView=Y',
 '/rocksoccer/ADs2/481917?searchView=Y',
 '/rocksoccer/ADs2/481915?searchView=Y',
 '/rocksoccer/ADs2/481721?searchView=Y',
 '/rocksoccer/ADs2/481536?searchView=Y',
 '/rocksoccer/ADs2/481478?searchView=Y',
 '/rocksoccer/ADs2/480980?searchView=Y',
 '/rocksoccer/ADs2/480804?searchView=Y',
 '/rocksoccer/ADs2/480802?searchView=Y',
 '/rocksoccer/ADs2/480725?searchView=Y',
 '/rocksoccer/AD